### Домашнее задание:

Задача: Поиск похожих по эмбеддингам.

1.На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу).

2.Проверить насколько хорошо работают подходы.

In [ ]:
import json
import string
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
!pip install stop_words
from stop_words import get_stop_words
!pip install annoy
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
def get_datasets(path):
  data = []
  with open(path, 'r') as r:
      for line in r:
          data.append(json.loads(line))
  data.sort(key=lambda x: x['date'])
  return data


def get_text(data):
  return [data['title'], data['text']]


def get_tokens(data, stop_words=None, morph_analyzer=None):

  text = ''.join(i for i in data.strip() if i not in punct).split()  
  tokens = [tkn.lower() for tkn in text if tkn not in stop_words]
  tokens = [morpher.parse(tkn)[0].normal_form for tkn in tokens if len(tkn) > 1]
  return tokens


def get_response(text, index, model, index_map):
    text = get_tokens(text)
    vector = np.zeros(300)
    norm = 0
    for word in text:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    titles = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in titles]


def read_article(article_name):
  for item in range(len(x_train)):
    if x_train[item]['title'] == article_name:
      return x_train[item]['text']

In [ ]:
!wget https://www.dropbox.com/s/lb50mk5jujjjqbi/gazeta_jsonl_v2.tar.gz
!tar -xvzf gazeta_jsonl_v2.tar.gz

In [5]:
train_path = 'gazeta_train.jsonl'
x_train = get_datasets(train_path)

print(f'Train length: {len(x_train)}')


Train length: 60964


In [6]:
x_train[1]

{'date': '2010-06-01 10:42:59',
 'url': 'https://www.gazeta.ru/culture/2010/06/01/a_3378006.shtml',
 'summary': 'Британские затворники, московские модники, бразильский фанк и исламский панк, а также Мик Джаггер, группа Blur и фестивальные хроники, снятые на мобильник, – в Москве стартует Beat Film Festival.',
 'title': 'Секс, наркотики и темный зал',
 'text': "Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий 

In [7]:
x_train[1]['text']

"Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий в этот четверг Beat Film Festival – прямой наследник Music.doc. На этот раз к услугам киномеломанов самая авторитетная арт-хаусная площадка города, а не тесный клуб или помпезный «Иллюзион», а вместо собранной по сусекам программы – фильмы с не более чем двухлетним сроком давности. Картины «Bassweight» и «Favellla on Blast» рассказывают (соответственно) о все 

In [10]:
articles = []
for line in tqdm_notebook(x_train):
  articles.append(get_text(line))
  if len(articles) > 10000: 
    break

  0%|          | 0/60964 [00:00<?, ?it/s]

In [12]:
articles[0]

['Налог в бак',
 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодн

In [13]:
morpher = MorphAnalyzer()
stop_words = get_stop_words('ru')
punct = list(string.punctuation + '—' + '–')

In [15]:
tokens = []
for artcl in tqdm_notebook(articles):
  temp = []
  for item in artcl:    
    temp += get_tokens(item, stop_words, morpher)
  tokens.append(temp) 
  # if len(articles) > 2:
  #   break

  0%|          | 0/10001 [00:00<?, ?it/s]

In [18]:
tokens[140][:10]

['renaultnissan',
 'корейский',
 'кухня',
 'альянс',
 'renault',
 'nissan',
 'официально',
 'подтвердить',
 'заинтересованность',
 'приобретение']

In [19]:
modelW2V = Word2Vec(sentences=tokens, size=300, min_count=1, window=5)
modelFT = FastText(sentences=tokens, size=300, min_count=1, window=5)

In [23]:
print('музычка' in modelW2V.wv)
print('музычка' in modelFT.wv)

False
True


In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')
index_map = {}
counter = 0

for line in tqdm_notebook(articles):
        n_w2v = 0
        n_ft = 0  

        index_map[counter] = line[0]    

        article_body = get_tokens(line[1])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)        

        for word in article_body:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]                
                n_w2v += 1  
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft

        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 10000:
            break

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/10001 [00:00<?, ?it/s]

True

### Проверка корректности результатов:

In [ ]:
TEXT = 'транспорт'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

['Светофоры поумнеют', 'Мэрия подавляет коллапс интеллектом ', 'Шереметьево поедет через область', 'Пробка подходит к развязке', 'Москва расколется по линии моста']
['Москва для пешеходов', 'Светофоры поумнеют', 'Фуры увязли в весне', 'Государство привьет транспорту интеллект', 'Иркутск не дотянул до нового аэропорта']


Вывод:
- Word2Vec: тематика статей корректна по отношению к запросу;
- FastText: тематика статей корректна по отношению к запросу.

In [ ]:
TEXT = 'налоговый сбор'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

['Не как народ', 'Чиновников больше, зарплаты выше', 'Обама-писатель богаче Обамы-президента', 'Бюджет не справился с нулем', '«Для других аппетитов средств нет»']
['ОСАГО перевалило за кризис', 'Авиапассажиры пополнят казну Германии', 'Частичное экономическое затмение', 'Власть увлеклась кризисом', 'Авиапассажиры пополнят казну Германии']


Вывод:
- Word2Vec: тематика статей корректна по отношению к запросу;
- FastText: тематика статей не совсем корректна по отношению к запросу (одна статья повторяется 2 раза).

In [ ]:
TEXT = 'искусственный интеллект'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

['Так делать дороги нельзя ни в коем случае', 'Porsche в зоне рулежки', 'Застывшее движение', 'Мал золотник, да дорог', 'Стамбул колоссальный']
['Manfrotto STILE — функциональность в итальянском стиле', 'Объем важен не только для прически', 'Нам нужны гаражи, а им – только наши деньги', 'Мал золотник, да дорог', 'Книги в интернете: скачать, прочесть, купить']


Вывод:
- Word2Vec: тематика статей некорректна по отношению к запросу;
- FastText: тематика статей некорректна по отношению к запросу.

In [ ]:
TEXT = 'про футбол'
print(get_response(TEXT, w2v_index, modelW2V, index_map))
print(get_response(TEXT, ft_index, modelFT, index_map))

['«Думаю, в Катаре будет спокойно»', '«Мы бежим впереди паровоза»', '«Спаллетти может тренировать сборную»', '«Овечкин расколол НХЛ»', '«Любите своих футболистов»']
['«Селтик» без Макгиди', 'Невилл ушел на пенсию', 'Пареха вместе с Месси', 'Давидс вернулся в большой футбол', 'Жирков ждет Акинфеева']


Вывод:
- Word2Vec: тематика статей корректна по отношению к запросу;
- FastText: тематика статей корректна по отношению к запросу.

### ВЫВОД:

В данном случае более корректно справился Word2Vec (тематики статей боллее ближе соответсвовали запросу).

Для повышения точности можно попробовать скорректировать ряд параметров:
- Увеличить кол-во статей;
- Кол-во центроид;
- Длину вектора эмбеддинга.